# Regression logistique

In [104]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd

app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
    train_labels = app_train['TARGET']
else:
    train = app_train.copy()

# Séparer les colonnes numériques et non numériques
numeric_cols = train.select_dtypes(include=['number']).columns
categorical_cols = train.select_dtypes(exclude=['number']).columns

# Imputation des valeurs manquantes pour les colonnes numériques
imputer = SimpleImputer(strategy='median')
train_numeric = imputer.fit_transform(train[numeric_cols])
test_numeric = imputer.transform(app_test[numeric_cols])

# Normalisation des colonnes numériques
scaler = MinMaxScaler(feature_range=(0, 1))
train_numeric = scaler.fit_transform(train_numeric)
test_numeric = scaler.transform(test_numeric)

# Pour simplifier, on ignore les colonnes catégorielles ici
# Vous pouvez appliquer un encodage (OneHotEncoder) si besoin
train_final = pd.DataFrame(train_numeric, columns=numeric_cols)
test_final = pd.DataFrame(test_numeric, columns=numeric_cols)

# Créer et entraîner le modèle
log_reg = LogisticRegression(C=0.0001)
log_reg.fit(train_final, train_labels)

# Faire des prédictions
log_reg_pred = log_reg.predict_proba(test_final)[:, 1]

# Créer le DataFrame de soumission
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()


Training data shape:  (215257, 105)
Testing data shape:  (92254, 105)


,SK_ID_CURR,TARGET
0,342180,0.081685
1,259636,0.064739
2,305882,0.077060
3,243264,0.063606
4,264946,0.064830


# Random Forest

In [134]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Charger les données
app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Convertir toutes les colonnes en numériques en utilisant LabelEncoder
label_encoders = {}
for col in app_train.columns:
    if app_train[col].dtype == 'object':
        le = LabelEncoder()
        app_train[col] = le.fit_transform(app_train[col].astype(str))
        if col in app_test.columns:
            app_test[col] = le.transform(app_test[col].astype(str))
        label_encoders[col] = le

# Séparer les données d'entraînement et les labels
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
    train_labels = app_train['TARGET']
else:
    train = app_train.copy()
    train_labels = None

# Préparer les données de test sans l'identifiant
test = app_test.drop(columns=['SK_ID_CURR'])

# S'assurer que les colonnes sont alignées avant l'imputation
common_cols = train.columns.intersection(test.columns)
train = train[common_cols]
test = test[common_cols]

# Imputer les valeurs manquantes avec la médiane
imputer = SimpleImputer(strategy='median')
train = imputer.fit_transform(train)
test = imputer.transform(test)

# Créer le modèle RandomForest
random_forest = RandomForestClassifier(n_estimators=100, random_state=50, verbose=1, n_jobs=-1)

# Entraîner le modèle
random_forest.fit(train, train_labels)

# Extraire l'importance des caractéristiques
feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': common_cols, 'importance': feature_importance_values})

# Faire des prédictions
predictions = random_forest.predict_proba(test)[:, 1]

# Créer le DataFrame de soumission
submit = pd.DataFrame({'SK_ID_CURR': app_test['SK_ID_CURR'], 'TARGET': predictions})

print("Prédictions:")
print(submit.head())


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.9s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.2s finished


Prédictions:
   SK_ID_CURR  TARGET
0      342180    0.18
1      259636    0.05
2      305882    0.10
3      243264    0.03
4      264946    0.04
